In [ ]:
# determine percent changes 
# things to optimize
    # R value 
        # Roof 
        # Wall
        # Floor 
    # Window
        # Shgc 
        # U-Factor
    # Lighting 
    # Daylight sensors 
    # Equipment 
    # Setpoints -> consider pmv as a constraint 


# calculate energy savings 
# think about npv / payback period to determine max decent const cost 


In [20]:
import numpy as np
import os
from eppy import *
from eppy.modeleditor import IDF

In [23]:
opt_models_dir = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/opt_models"


# set the idd
iddfile = "/Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd"
IDF.setiddname(iddfile)

# get the eopw
epw = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw"

In [30]:
root_idf_path = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/calibration/eppy_adjusted_models/02_28-1_Palo_Alto_EPW/in2.idf"
root_idf = IDF(root_idf_path, epw)

# Helper Functions

In [45]:
def perc_change_val(pc, xo):
    "given percent change pc, and xo, calc x_new, xn"
    xn = (xo*pc)/100 + xo
    xn_real = max(0, xn)
    return xn_real

def perc_change_arr(pc_arr, xo):
    "given array of percent changes pc_arr, and xo, calc x_new_arr, xn_arr"
    xn_arr = [perc_change_val(pc, xo) for pc in pc_arr]
    return xn_arr
        
# generate new idf 
def new_idfs(opt_group_name, idf_adjust_fx):
    # make a group folder
    group_folder = os.path.join(opt_models_dir, opt_group_name)
    os.makedirs(group_folder)
    # keep track of the values changed in the idf 
    new_vals = {}
    for ix, pc in enumerate(pc_arr):
        # make a folder for the specific run 
        run_folder = os.path.join(group_folder, f"run_{pc}")
        os.makedirs(run_folder)
        # make a copy of an og idf
        idf0_path = os.path.join(run_folder, "opt.idf")
        root_idf.save(idf0_path)
        # initialize this new idf 
        idf0 = IDF(idf0_path, epw)

        # call function to modify the idf 
        idf0, new_val = idf_adjust_fx(idf0, pc)
        # save the idf here 
        idf0.save()
        # save the new vals
        new_vals[pc] = new_val
    print(group_folder)
    print(new_vals)
    # TODO save to csv in group folder 
    
    return group_folder


# run idfs 
def run_idfs(group_path):
    print(group_path)
    sub_dirs = os.listdir(group_path)
    for sub_dir in sub_dirs:
        dir = os.path.join(group_path, sub_dir)
        print(sub_dir)
        idf_path = os.path.join(dir, "opt.idf")
        idf0 = IDF(idf_path, epw)
        idf0.run(output_directory = dir)



# extract relevant data 
# create dictionary of runs and their names 
# pass to calibration functions like thing 
# just care about hot water, elect, chilled water, and their tarrifs
# inside folder, csv with the data 
# for a group, csv in group folder + csv in run folder with results -> another csv 
# all this in an executable 
# 
# analysis notebook 
# then can correlate energy use with cost, 
# as well as tornado diagrams for energy use and cost -> subplots
# can compare individual schemes to see which give greatest e savings / cost 
# can pair schemes to maximize.. 




# Optimization Functions

## Exterior Surface R-Values

## Window

In [25]:
def change_window_shgc(idf0, pc):
    window_mat = [s  for s in idf0.idfobjects["WindowMaterial:SimpleGlazingSystem"] if s.Name == "Thornton Window Material 1"][0]
    # get current value 
    curr_val = window_mat.Solar_Heat_Gain_Coefficient 
    print(curr_val)
    # calculate new values 
    new_val = perc_change_val(pc, curr_val)
    # adjust the idf 
    window_mat.Solar_Heat_Gain_Coefficient = new_val
    # return the idf
    print([s  for s in idf0.idfobjects["WindowMaterial:SimpleGlazingSystem"] if s.Name == "Thornton Window Material 1"][0])
    return (idf0, new_val)


In [39]:
pc_arr = [-10, 10]
a = new_idfs("test3", change_window_shgc)
run_idfs(a)

0.31

WindowMaterial:SimpleGlazingSystem,
    Thornton Window Material 1,    !- Name
    2.66878377032334,         !- UFactor
    0.279,                    !- Solar Heat Gain Coefficient
    0.42;                     !- Visible Transmittance

0.31

WindowMaterial:SimpleGlazingSystem,
    Thornton Window Material 1,    !- Name
    2.66878377032334,         !- UFactor
    3.410000e-01,             !- Solar Heat Gain Coefficient
    0.42;                     !- Visible Transmittance

/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/opt_models/test3


/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/opt_models/test3
run_1

/Applications/OpenStudioApplication-1.1.1/EnergyPlus/energyplus --weather /Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw --output-directory /Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/opt_models/test3/run_1 --idd /Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd /Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/in.idf

EnergyPlus Starting
EnergyPlus, Version 9.4.0-998c4b761e, YMD=2022.03.11 06:02


Could not find platform independent libraries <prefix>
Could not find platform dependent libraries <exec_prefix>
Consider setting $PYTHONHOME to <prefix>[:<exec_prefix>]


Initializing Response Factors
Calculating CTFs for "ASHRAE 90.1-2010 EXTROOF IEAD CLIMATEZONE 2-8"
Calculating CTFs for "ASHRAE 90.1-2010 EXTWALL MASS CLIMATEZONE 3"
Calculating CTFs for "EXTSLABCARPET 4IN CLIMATEZONE 1-8"
Calculating CTFs for "INTERIOR CEILING"
Calculating CTFs for "INTERIOR WALL"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
In

EnergyPlus Completed Successfully.



/Applications/OpenStudioApplication-1.1.1/EnergyPlus/energyplus --weather /Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw --output-directory /Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/opt_models/test3/run_0 --idd /Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd /Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/in.idf

EnergyPlus Starting
EnergyPlus, Version 9.4.0-998c4b761e, YMD=2022.03.11 06:03


Could not find platform independent libraries <prefix>
Could not find platform dependent libraries <exec_prefix>
Consider setting $PYTHONHOME to <prefix>[:<exec_prefix>]


Initializing Response Factors
Calculating CTFs for "ASHRAE 90.1-2010 EXTROOF IEAD CLIMATEZONE 2-8"
Calculating CTFs for "ASHRAE 90.1-2010 EXTWALL MASS CLIMATEZONE 3"
Calculating CTFs for "EXTSLABCARPET 4IN CLIMATEZONE 1-8"
Calculating CTFs for "INTERIOR CEILING"
Calculating CTFs for "INTERIOR WALL"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
In

EnergyPlus Completed Successfully.


In [ ]:
window_mat = [s  for s in idf0.idfobjects["WindowMaterial:SimpleGlazingSystem"] if s.Name == "Thornton Window Material 1"][0]
window_mat.Solar_Heat_Gain_Coefficient = 0.25 # prev 0.31
window_mat.UFactor = 1.20